In [51]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### K-Means para aprendizaje semi-supervisado con dataset ISOLET

In [52]:
# Cargar dataset ISOLET
df = pd.read_csv(r"m:\UNIVERSIDAD 2025_2\INTELIGENCIA ATRIFICIAL\NoSupervisado\Dataset\ISOLET_2.csv", 
                 header=None, skiprows=1)

X = df.iloc[:, :-1].values  # características de audio
y = df.iloc[:, -1].str.strip("'").astype(int)  # etiquetas (1-26 = A-Z)

print("Shape de X:", X.shape)
print("Cantidad de clases:", len(np.unique(y)))

Shape de X: (7797, 617)
Cantidad de clases: 26


In [53]:
# Dividimos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
k = 50  # número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
X_audio_dist = kmeans.fit_transform(X_train)

print("Shape de distancias:", X_audio_dist.shape)

Shape de distancias: (5847, 50)


In [ ]:
idxs = np.argmin(X_audio_dist, axis=0)  # índice más cercano a cada centroid
X_representative = X_train[idxs]  # muestras representativas

print("Shape:", X_representative.shape)

Shape: (50, 617)


In [66]:
# Obtener las etiquetas reales de las muestras más representativas
y_representative = y_train.iloc[idxs]
print(y_representative)

1797    15
5825     3
6573    13
3490     7
6230    19
1334    18
3081     7
6391    25
4271     8
2466    12
3890    17
5998    21
1224    15
5096     1
7175     2
3565     4
3700     9
3696     5
6865     3
6953    21
1995    10
3709    18
5119    24
1345    23
6041    12
6030     1
720     23
4287    24
2723    10
6371    15
6723    10
750     12
6478    17
2394     2
6132    25
3321    20
5446    14
4567    18
1031    22
7106    19
6483    20
3536     1
3650    11
1534    14
4904    17
6356     8
2455     6
3554    19
2261    13
3716    25
Name: 617, dtype: int32


In [ ]:
log_reg2 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg2.fit(X_representative, y_representative)

# Evaluamos el modelo en el conjunto de prueba
print("Precisión usando solo muestras representativas:", log_reg2.score(X_test, y_test))

Precisión usando solo muestras representativas: 0.6353846153846154


In [ ]:
log_reg_random = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg_random.fit(X_train[:50], y_train[:50])


# Evaluamos el modelo en el conjunto de prueba
print("Precisión usando 50 muestras aleatorias:", log_reg_random.score(X_test, y_test))

Precisión usando 50 muestras aleatorias: 0.48307692307692307


In [61]:
# Propagación de etiquetas representativas a todo el conjunto de entrenamiento
y_train_propagated = np.empty(len(X_train))
for i in range(k):
    y_train_propagated[kmeans.labels_ == i] = y_representative.iloc[i]

# Entrenamiento con las etiquetas propagadas
log_reg3 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg3.fit(X_train[:1000], y_train_propagated[:1000])

print("Precisión con 1000 muestras semi-etiquetadas:", log_reg3.score(X_test, y_test))

Precisión con 1000 muestras semi-etiquetadas: 0.66


## Aprendizaje Activo

In [62]:
# Calculamos las probabilidades de las primeras 1000 muestras
probas = log_reg3.predict_proba(X_train[:1000])

# Obtenemos el índice de la clase más probable para cada muestra
labels_ixs = np.argmax(probas, axis=1)

# Obtenemos la confianza (probabilidad máxima) de cada predicción
confidences = probas[np.arange(len(probas)), labels_ixs]

In [64]:
# Ordenamos por confianza (de menor a mayor)
sorted_ixs = np.argsort(confidences)

k_active = 50  # cantidad de muestras "inciertas" a revisar

# Obtenemos las 50 muestras más inciertas
X_lowest = X_train[:1000][sorted_ixs[:k_active]]
y_lowest = y_train.iloc[:1000].iloc[sorted_ixs[:k_active]]

In [65]:
log_reg_active = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)

# Entrenamos con las 50 muestras inciertas (suponemos que ya están etiquetadas correctamente)
log_reg_active.fit(X_lowest, y_lowest)

# Evaluamos en el conjunto de prueba
print("Precisión usando aprendizaje activo (50 muestras inciertas):", log_reg_active.score(X_test, y_test))

Precisión usando aprendizaje activo (50 muestras inciertas): 0.3035897435897436
